<a href="https://colab.research.google.com/github/Mohammadhsiavash/DeepL-Training/blob/main/AI%20Agents%20%2B%20Automation/Math_Reasoning_ReAct_Patern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create an LLM-powered agent that can solve math word problems by:
Reasoning through intermediate steps (thinking)
Acting via a calculator tool (e.g., eval or Python)
Returning a final answer

Define the ReAct Prompt Template

In [1]:
REACT_PROMPT = """You are a Math Reasoning Agent. You solve problems by reasoning about the problem step by step.
Use this format:
Question: <question>
Thought: <your reasoning>
Action: <python code>
Observation: <result>
... (repeat as needed)
Final Answer: <answer>
Question: What is 12 divided by 3 and then multiplied by 4?
Thought: First divide 12 by 3 to get 4, then multiply by 4.
Action: 12 / 3
Observation: 4.0
Action: 4.0 * 4
Observation: 16.0
Final Answer: 16
Question: {question}
"""


 Implement the Reasoning + Action Loop

In [2]:
import re

def evaluate_code(code):
    try:
        result = eval(code)
        return result
    except Exception as e:
        return f"Error: {str(e)}"
def react_loop(question, llm):
  prompt = REACT_PROMPT.format(question=question)
  while True:
    response = llm(prompt)
    print(response)

    # Extract last action
    action_match = re.search(r'Action: (.*)', response)
    if not action_match:
        break
    else:
        action = action_match.group(1).strip()
        result = evaluate_code(action)
        prompt += f"Observation: {result}\n"

    if "Final Answer:" in response:
      break

    # Add model's reasoning step to the prompt
    prompt += f"\n{response.splitlines()[-1]}"

  return response

Use OpenAI or Local Model to Simulate Agent

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re

# Load the tokenizer and the model
model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

def call_llm(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    # Combine thinking content and content for the react loop
    # You might need to adjust this based on how the model structures its output
    # and how the react_loop expects the response.
    # For now, let's return both, perhaps formatted.
    return f"Thought: {thinking_content}\n{content}"


# Try the ReAct loop
question = "A rectangle has length 8 and width 5. What is its area?"
print(react_loop(question, call_llm))

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Thought: <think>
Okay, let's see. The question is asking for the area of a rectangle with length 8 and width 5. Hmm, area of a rectangle... I remember that formula is length multiplied by width. So, if the length is 8 and the width is 5, then the area should be 8 times 5. Let me check that again. Multiplying them gives 40. So the answer should be 40. But wait, maybe I should write it as a fraction to be precise? No, 8 times 5 is 40, so that's straightforward. I don't think there's anything else to it here. The steps are simple: multiply the two numbers. Yep, 8 * 5 is 40. So the final answer is 40.
</think>
Thought: The area of a rectangle is calculated by multiplying its length and width.  
Action: 8 * 5  
Observation: 40  
Final Answer: 40
Thought: <think>
Okay, let's see. The question is asking for the area of a rectangle with length 8 and width 5. Hmm, area of a rectangle... I remember that formula is length multiplied by width. So, if the length is 8 and the width is 5, then the ar